In [1]:
from hhr_parsing_functions import *
from network_creation_functions import *
import subprocess
from collections import defaultdict

### Search Parsing

##### hhsearch


In [9]:
input_dir='./search_outputs_range/second_search/hits_hhm'
output_dir='./search_outputs_range/sthird_search/not_parsed_hhr'
subprocess.run(["./hhm_search.sh", input_dir, output_dir])

Input dir: ./search_outputs_range/second_search/hits_hhm
Output dir: ./search_outputs_range/sthird_search/not_parsed_hhr
Starting processing of 844 files...
Searching COG0072_250-335 (% complete).
Searching COG0188_20-114 (.100% complete).
Searching COG0217_29-95 (.200% complete).


In [8]:
# For Background Search
input_dir='./search_outputs_range/first_search/hits_hhm'
output_dir='./search_outputs_range/second_search/not_parsed_hhr'
log_file = open('./search_outputs_range/second_search_range_nohup.log', 'w')
process = subprocess.Popen(["nohup", "./hhm_search.sh", input_dir, output_dir],
                           stdout=log_file, stderr=subprocess.STDOUT)

##### Profile Extraction

In [ ]:
## ## EXTRACTION OF HITTED .hhm

# Directory containing output files from HHsearch.
hhsearch_output_dir = "/home/nicola/internship/winghel/network/search_outputs/first_search/not_parsed_hhr"
list_of_hits = []

# Iterate over each HHsearch output file to extract and compile a list of unique hits in the fisrt search
for filename in os.listdir(hhsearch_output_dir):
    hhsearch_output_file=os.path.join(hhsearch_output_dir, filename) # Create path to file
    new_hits=extract_hits_names(hhsearch_output_file, p_cutoff=20) # Extract the hits name
    unique_new_hits = list(set(new_hits)) # Removes duplicates from the list of new hits.
    list_of_hits.extend(new_hits) # Adds the new hits to the cumulative list of hits.


previous_search_queries=[file.split('.')[0] for file in os.listdir(hhsearch_output_dir)]
print(f"In total there are {len(set(list_of_hits))} unique hits, before removing the already searched queries")
list_of_hits = list(set(list_of_hits)-set(previous_search_queries)) # Removes duplicates from the previous search
print(f"In total there are {len(list_of_hits)} unique hits")


# Directory to save the extracted HMM profiles.
save_directory="/home/nicola/internship/winghel/network/search_outputs/first_search/hits_hhm"

# Extracts and saves the HMM profiles for the list of unique hits.
extract_hmm_profiles(list_of_hits, save_directory, database_path="/home/nicola/internship/hhsuite/databases/COG_KOG/COG_KOG_hhm.ffdata")


In [2]:
adjusted_ranges_mille = defaultdict(list)

In [3]:
## EXTRACTION OF HITS -- CUTTED PROFILES

# Directory containing output files from HHsearch.
hhsearch_output_dir = "/home/nicola/internship/winghel/network/search_outputs_range/2_search/not_parsed_hhr"
# hhsearch_output_dir=output_dir
hits_dict = defaultdict(list)

# Iterate over each HHsearch output file to extract and compile a list of unique hits in the fisrt search
for filename in os.listdir(hhsearch_output_dir):
    hhsearch_output_file=os.path.join(hhsearch_output_dir, filename) # Create path to file
    hits_dict=extract_hits_names_and_range(hhsearch_output_file, hits_dict, p_cutoff=40, cols_cutoff=30) # Extract the hits name

previous_search_queries=[file.split('.')[0].split('_')[0] for file in os.listdir(hhsearch_output_dir)]
print(f"In total there are {len(hits_dict)} unique hits, before removing the already searched queries")
hits_dict={key: value for key, value in hits_dict.items() if key not in previous_search_queries} # Removes duplicates from the previous search
print(f"In total there are {len(hits_dict)} unique hits")

# Process the ranges
processed_ranges = process_ranges(hits_dict)
adjusted_ranges = adjust_ranges(processed_ranges)


In total there are 452 unique hits, before removing the already searched queries
In total there are 292 unique hits


In [4]:
counter=0
for list in adjusted_ranges.values():
    for range in list:
        counter += 1
counter

print(f"In total there are {len(hits_dict)} unique hits, hitted in {counter} different ranges")


331

In [27]:
## EXTRACTION OF .hhm -- CUTTED PROFILES
# Directory to save the extracted HMM profiles.
save_directory="/home/nicola/internship/winghel/network/search_outputs_range/3_search/hits_hhm"

# Extracts and saves the HMM profiles for the list of unique hits.
extract_hmm_profiles_and_cut_range(adjusted_ranges, save_directory, database_path="/home/nicola/internship/hhsuite/databases/COG_KOG/COG_KOG_hhm.ffdata")



Processing file: 100%|██████████| 12324031/12324031 [00:13<00:00, 884490.69it/s]


##### Parse .hhs

In [2]:
## REMOVE SELF HITS
list_of_searched_profiles=[]

for search in [1, 2, 3]:
    # Directory containing the initial HHsearch output files.
    path=f'/home/nicola/internship/winghel/network/search_outputs_range/{search}_search/not_parsed_hhr'

    # Directory to save the processed HHsearch output, with substitutions applied.
    save_dir=f'/home/nicola/internship/winghel/network/search_outputs_range/{search}_search/parsed_hhr'
    
    # Process each file: remove self-hits.
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        save_path = os.path.join(save_dir, file)
        remove_self_hit(file_path, save_path)

### Network

In [3]:
# Get all the PROFILE_RANGE searched
first_search_hits=[(file.split('.')[0].split('_')[0],file.split('.')[0].split('_')[1])  for file in os.listdir('/home/nicola/internship/winghel/network/search_outputs_range/2_search/not_parsed_hhr')]
second_search_hits=[(file.split('.')[0].split('_')[0],file.split('.')[0].split('_')[1]) for file in os.listdir('/home/nicola/internship/winghel/network/search_outputs_range/3_search/not_parsed_hhr')]
range_search_hits=first_search_hits+second_search_hits

range_search_hits_dict = defaultdict(list)

for tuple in range_search_hits:
    profile = tuple[0]
    # start, end = map(int, tuple[1].split('-'))
    range_search_hits_dict[profile].append(tuple[1])


print(f'There are {len(range_search_hits)} WHD hitted in {len(range_search_hits_dict)} different proteins') # It contains the query proteins, with their searched range

There are 495 WHD hitted in 444 different proteins


In [4]:
### NETWORK CREATION

# Directories containing parsed HHsearch output files for network creation.
hhsearch_output_dirs = ["/home/nicola/internship/winghel/network/search_outputs_range/1_search/parsed_hhr",
                            "/home/nicola/internship/winghel/network/search_outputs_range/2_search/parsed_hhr",
                                "/home/nicola/internship/winghel/network/search_outputs_range/3_search/parsed_hhr"]

# Initialize the hits list with headers for the network edges table.
edges_list = [] # Initialize the hits lists
edges_list.append("Query\tHit\tProb\tE_value\tP_value\tScore\tSS\tCols\tQuery_HMM\tTemplate_HMM\tEdge_type")

# Iterate over each directory to extract edges from HHsearch output files.
for directory in hhsearch_output_dirs:
    for filename in os.listdir(directory):
        file_path=os.path.join(directory, filename) # Create path to file    
        # Extract edges with specified cutoffs for E-value and alignment length from the file.    
        edges_list = get_edges_from_file_cut_off_and_range(file_path, filename, edges_list, E_cut_off=40, min_alig_len=30, Prob_cut_off = 40) # Extract

print(len(edges_list))


35341


In [5]:
df_network = pd.DataFrame([x.split('\t') for x in edges_list[1:]], columns=edges_list[0].split('\t'))
df_network

,Query,Hit,Prob,E_value,P_value,Score,SS,Cols,Query_HMM,Template_HMM,Edge_type
0,ask1,KOG3433_9-70,96.8,0.0025,2.6E-07,39.0,7.7,59,5-67,9-70,BH
1,ask1,KOG4603_8-68,96.6,0.0044,4.5E-07,37.8,7.6,58,6-67,8-68,SBH
2,ask1,COG5124_10-71,96.5,0.0055,5.7E-07,37.8,7.6,58,5-67,10-71,H
3,ask1,COG1510_40-77,95.3,0.037,3.9E-06,31.4,7.2,38,23-61,40-77,H
4,ask1,COG1378_20-63,95.3,0.025,2.6E-06,33.0,6.5,44,9-58,20-63,H
...,...,...,...,...,...,...,...,...,...,...,...
35335,KOG4772_107-181,COG2162_51-88,42.4,13,0.0014,21.6,4.3,34,30-63,51-88,H
35336,KOG4796_197-276,KOG1140_776-846,96.7,0.00049,5E-08,53.4,4.1,64,11-77,776-846,BH
35337,KOG4796_197-276,KOG1139_399-473,96.2,0.002,2.1E-07,46.2,5.0,64,12-77,399-473,SBH
35338,KOG4796_197-276,COG2137_17-61,88.2,0.57,5.8E-05,26.9,6.0,45,11-58,17-61,H


In [ ]:
df_network_updated = update_dataframe_ranges(df_network, range_search_hits_dict)

# Full path to the directory where the network edge file will be saved.
full_path = "/home/nicola/internship/winghel/network/network"
network_file_path = os.path.join(full_path, "network_total.txt")

df_network_updated.to_csv(network_file_path, sep='\t', index=False)
df_network_updated

In [ ]:
df_network[df_network['Query'].str.contains('COG1429', na=False)]

In [ ]:
df_network_updated[df_network_updated['Query'].str.contains('COG1429', na=False)]

In [ ]:
n_query_1=(set(df_network['Query']))
n_target_1=(set(df_network['Hit']))

In [ ]:
n_query_2=(set(df_network_updated['Query']))
n_target_2=(set(df_network_updated['Hit']))